In [1]:
import pandas as pd
import numpy as np


In [2]:
from tesi.database.di import get_session_maker
from tesi.zappai.di import (
    get_cds_api,
    get_climate_generative_model_repository,
    get_future_climate_data_repository,
    get_location_repository,
    get_past_climate_data_repository,
)
from tesi.zappai.repositories.dtos import ClimateDataDTO, FutureClimateDataDTO
from tesi.zappai.utils import common


session_maker = get_session_maker()
cds_api = get_cds_api()
location_repository = get_location_repository(session_maker=session_maker)
past_climate_data_repository = get_past_climate_data_repository(
    session_maker=session_maker,
    cds_api=cds_api,
    location_repository=location_repository,
)
future_climate_data_repository = get_future_climate_data_repository(
    session_maker=session_maker, cds_api=cds_api
)
climate_generative_model_repository = get_climate_generative_model_repository(
    session_maker=session_maker,
    location_repository=location_repository,
    past_climate_data_repository=past_climate_data_repository,
    future_climate_data_repository=future_climate_data_repository,
)

location = await location_repository.get_location_by_country_and_name(
    country=common.EXAMPLE_LOCATION_COUNTRY, name=common.EXAMPLE_LOCATION_NAME
)
if location is None:
    location = await location_repository.create_location(
        country=common.EXAMPLE_LOCATION_COUNTRY,
        name=common.EXAMPLE_LOCATION_NAME,
        longitude=common.EXAMPLE_LONGITUDE,
        latitude=common.EXAMPLE_LATITUDE,
    )
past_climate_data_df = ClimateDataDTO.from_list_to_dataframe(
    await past_climate_data_repository.get_past_climate_data(location.id)
)
future_climate_data_df = FutureClimateDataDTO.from_list_to_dataframe(
    await future_climate_data_repository.get_future_climate_data_for_nearest_coordinates(
        longitude=location.longitude,
        latitude=location.latitude,
        start_year=1970,
        start_month=1,
    )
)

await climate_generative_model_repository.delete_climate_generative_model(
    location_id=location.id
)
climate_generative_model, x_df_test, y_df_test = (
    await climate_generative_model_repository.create_model_for_location(
        location.id
    )
)
model = climate_generative_model.model
x_scaler = climate_generative_model.x_scaler
y_scaler = climate_generative_model.y_scaler


2024-06-10 23:53:18.345805: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 23:53:18.348163: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 23:53:18.380518: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 23:53:19.064583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/50


2024-06-10 23:53:20.890107: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-10 23:53:20.890583: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 1.1487 - root_mean_squared_error: 1.0712 - val_loss: 0.8313 - val_root_mean_squared_error: 0.9118
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0024 - root_mean_squared_error: 1.0007 - val_loss: 0.7667 - val_root_mean_squared_error: 0.8756
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9259 - root_mean_squared_error: 0.9619 - val_loss: 0.7082 - val_root_mean_squared_error: 0.8415
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8273 - root_mean_squared_error: 0.9094 - val_loss: 0.6541 - val_root_mean_squared_error: 0.8088
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7970 - root_mean_squared_error: 0.8922 - val_loss: 0.6037 - val_root_mean_squared_error: 0.7770
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7938 - root_mean_squared_error: 0.8898 - val_loss: 0.5541 - val_root_mean_squared_error: 0.7444
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7689 - root_me

In [3]:
from typing import cast


x_test_scaled = cast(np.ndarray, x_scaler.transform(x_df_test))
y_test_scaled = cast(np.ndarray, y_scaler.transform(y_df_test))

In [4]:
SEQ_LENGTH = 12

In [5]:
generated_data = await climate_generative_model_repository.generate_climate_data_from_last_past_climate_data(
    location_id=location.id # type: ignore
)
generated_data

(55, 17)
(55, 10)


surface_net_thermal_radiation  volumetric_soil_water_layer_3  \
year month                                                                 
2024 6                         -7874871.0                       0.122020   
     7                         -8926489.0                       0.127585   
     8                         -8975105.0                       0.127361   
     9                         -8991709.0                       0.127426   
     10                        -8992670.0                       0.127423   
     11                        -8992801.0                       0.127422   
     12                        -8992818.0                       0.127422   
2025 5                         -8747173.0                       0.132860   
     6                         -8747173.0                       0.132860   
     7                         -8747173.0                       0.132860   
     8                         -8747173.0                       0.132860   
     9                         -8809763.0                       0.133451   
     10                        -9063525.0                       0.140624   
     11                        -9063525.0                       0.140624   
     12                        -9063525.0                       0.140624   
2026 5                         -9063525.0                       0.140624   
     6                         -9063525.0                       0.140624   
     7                         -9063525.0                       0.140624   
     8                         -9063525.0                       0.140624   
     9                         -9063525.0                       0.140624   
     10                        -9063525.0                       0.140624   
     11                        -9063525.0                       0.140624   
     12                        -9063525.0                       0.140624   
2027 5                         -9063525.0                       0.140624   
     6                         -9063525.0                       0.140624   
     7                         -9063525.0                       0.140624   
     8                         -9063525.0                       0.140624   
     9                         -9063525.0                       0.140624   
     10                        -9063525.0                       0.140624   
     11                        -9063525.0                       0.140624   
     12                        -9063525.0                       0.140624   
2028 5                         -9063525.0                       0.140624   
     6                         -9063525.0                       0.140624   
     7                         -9063525.0                       0.140624   
     8                         -9063525.0                       0.140624   
     9                         -9063525.0                       0.140624   
     10                        -9063525.0                       0.140624   
     11                        -9063525.0                       0.140624   
     12                        -9063525.0                       0.140624   
2029 5                         -9063525.0                       0.140624   
     6                         -9063525.0                       0.140624   
     7                         -9063525.0                       0.140624   
     8                         -9063525.0                       0.140624   
     9                         -9063525.0                       0.140624   
     10                        -9063525.0                       0.140624   
     11                        -9063525.0                       0.140624   
     12                        -9063525.0                       0.140624   
2030 5                         -9063525.0                       0.140624   
     6                         -9063525.0                       0.140624   
     7                         -9063525.0                       0.140624   
     8                         -9063525.0                   

In [6]:
filtered_past_climate_data = past_climate_data_df[get_features()]
filtered_future_climate_data_df = future_climate_data_df.drop(
    columns=["latitude", "longitude"], errors="ignore"
)
# start generating from 1 year ago
seed_data = filtered_past_climate_data[(-12 - SEQ_LENGTH) : -12]
start_year, start_month = seed_data.index[-1]

filtered_future_climate_data_df = future_climate_data_df[
    ((future_climate_data_df.index.get_level_values("year") > start_year))
    | (
        (future_climate_data_df.index.get_level_values("year") == start_year)
        & (future_climate_data_df.index.get_level_values("month") > start_month)
    )
]

# this generated data start from one year ago
generated_data_df = generate_data_from_seed(
    model=model,
    x_scaler=x_scaler,
    y_scaler=y_scaler,
    seed_data=seed_data.to_numpy(),
    future_climate_data_df=filtered_future_climate_data_df,
)
true_data_df = past_climate_data_df[-12:]

NameError: name 'get_features' is not defined

In [ ]:
def plot_values(df: pd.DataFrame, values: list[str]):
    df_reset = df.reset_index()
    df_reset["YYYY-MM"] = (
        df_reset["year"].astype(str) + "-" + df_reset["month"].astype(str).str.zfill(2)
    )
    df_reset.plot(
        x="YYYY-MM",
        y=values,
        marker="o",
    )
    plt.xlabel("Month")
    plt.ylabel("Values")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

In [ ]:
generated_data_df = generated_data_df[:12]
generated_data_df["actual_soil_temperature_level_3"] = true_data_df["soil_temperature_level_3"]
true_data_df["cmip5_2m_temperature"] = filtered_future_climate_data_df[:12]["2m_temperature"]
plot_values(df=generated_data_df, values=["soil_temperature_level_3", "actual_soil_temperature_level_3"])
plot_values(df=true_data_df, values=["2m_temperature", "cmip5_2m_temperature"])

In [ ]:
plot_values(df=total_df, values=["volumetric_soil_water_layer_3"])